# Gerador de texto de João Guimarães Rosa

In [1]:
import tensorflow as tf
import numpy as np
import os

## Lendo os dados

In [2]:
text = open("obras.txt", 'rb').read().decode(encoding='utf-8')
print(f'Tamanho do corpus: {len(text)} caracteres')

Tamanho do corpus: 4726513 caracteres


In [3]:
vocab = sorted(set(text))
print(f'{len(vocab)} caracteres únicos')

132 caracteres únicos


## Processando o texto
### Vetorizando o texto

Antes do treinamento, precisamos mapear as strings para uma representação numérica. Para isso criaremos duas tabelas de pesquisa: uma mapeando caracteres para números e outra para números para caracteres.

In [4]:
char2idx = {u:i for i, u in enumerate(vocab)}

idx2char = np.array(vocab)

text_as_int = np.array([char2idx[c] for c in text])

In [5]:
print('{')
for char,_ in zip(char2idx, range(20)):
    print('  {:4s}: {:3d},'.format(repr(char), char2idx[char]))
print('  ...\n}')

{
  '\n':   0,
  '\x0c':   1,
  ' ' :   2,
  '!' :   3,
  '"' :   4,
  '&' :   5,
  "'" :   6,
  '(' :   7,
  ')' :   8,
  '*' :   9,
  ',' :  10,
  '-' :  11,
  '.' :  12,
  '/' :  13,
  '0' :  14,
  '1' :  15,
  '2' :  16,
  '3' :  17,
  '4' :  18,
  '5' :  19,
  ...
}


In [6]:
print('{} ---- caracteres mapeados para int ---- > {}'.format(repr(text[:13]), text_as_int[:13]))

'Grande Sertão' ---- caracteres mapeados para int ---- > [ 33  73  56  69  59  60   2  45  60  73  75 102  70]


## A tarefa de previsão

Dado um caractere, ou uma sequência de caractere, qual é o próximo caractere mais provável? Esta é a tarefa para a qual estamos treinando o modelo. A entrada para o modelo será uma sequência de caracteres e  treinaremos o modelo para prever a saída - o caractere a seguir em cada timestep.

Vamos dividir o texto em sequências de exemplo. Cada sequência de entrada conterá caracteres seq_length do texto.

Para cada sequência de entrada, os targets correspondentes contêm o mesmo comprimento de texto, exceto deslocado um caractere para a direita.

In [7]:
seq_length = 100
examples_per_epoch = len(text)//(seq_length+1)

# Criando exemplos de treinamento e targets 
char_dataset = tf.data.Dataset.from_tensor_slices(text_as_int)

for i in char_dataset.take(6):
    print(idx2char[i.numpy()])

G
r
a
n
d
e


O método batch nos permite converter facilmente esses caracteres individuais em sequências do tamanho desejado.

In [8]:
sequences = char_dataset.batch(seq_length+1, drop_remainder=True)

for item in sequences.take(5):
    print(repr(''.join(idx2char[item.numpy()])))

'Grande Sertão: Veredas\n\n \n― Nonada. Tiros que o senhor ouviu foram de briga de homem não, Deus esteja'
'.\nAlvejei mira em árvore, no quintal, no baixo do córrego. Por meu acerto. Todo dia isso\nfaço,  gosto'
';  desde  mal  em  minha  mocidade.  Daí,  vieram  me  chamar.  Causa  dum\nbezerro! um bezerro branco'
', erroso, os olhos de nem ser ― se viu ―; e com máscara de\ncachorro.  Me  disseram;  eu  não  quis  a'
'vistar.  Mesmo  que,  por  defeito  como  nasceu,\narrebitado  de  beiços,  esse  figurava  rindo  fei'


In [9]:
def split_input_target(chunk):
    input_text = chunk[:-1]
    target_text = chunk[1:]
    return input_text, target_text

dataset = sequences.map(split_input_target)

In [10]:
for input_example, target_example in  dataset.take(1):
    print('Input data: ', repr(''.join(idx2char[input_example.numpy()])))
    print('Target data:', repr(''.join(idx2char[target_example.numpy()])))

Input data:  'Grande Sertão: Veredas\n\n \n― Nonada. Tiros que o senhor ouviu foram de briga de homem não, Deus estej'
Target data: 'rande Sertão: Veredas\n\n \n― Nonada. Tiros que o senhor ouviu foram de briga de homem não, Deus esteja'


Cada índice desses vetores é processado como uma etapa única. Para a entrada na etapa de tempo 0, o modelo recebe o índice para "G" e tenta prever o índice para "r" como o próximo caractere. No próximo timestep, ele faz a mesma coisa, mas a RNN considera o contexto da etapa anterior além do caractere de entrada atual.

In [11]:
for i, (input_idx, target_idx) in enumerate(zip(input_example[:5], target_example[:5])):
    print("Step {:4d}".format(i))
    print("  input: {} ({:s})".format(input_idx, repr(idx2char[input_idx])))
    print("  expected output: {} ({:s})".format(target_idx, repr(idx2char[target_idx])))

Step    0
  input: 33 ('G')
  expected output: 73 ('r')
Step    1
  input: 73 ('r')
  expected output: 56 ('a')
Step    2
  input: 56 ('a')
  expected output: 69 ('n')
Step    3
  input: 69 ('n')
  expected output: 59 ('d')
Step    4
  input: 59 ('d')
  expected output: 60 ('e')


In [12]:
# Batch size
BATCH_SIZE = 64

# Buffer size embaralha o nosso dataset 
BUFFER_SIZE = 10000

dataset = dataset.shuffle(BUFFER_SIZE).batch(BATCH_SIZE, drop_remainder=True)

### Construindo o modelo

Usamos tf.keras.Sequential para definir o modelo. Para este exemplo simples, três camadas são usadas para definir nosso modelo:

- tf.keras.layers.Embedding : A camada de entrada. Uma tabela de pesquisa treinável que mapeará os números de cada caractere para um vetor com dimensões embedding_dim ;

- tf.keras.layers.GRU : Um tipo de RNN com units=rnn_units tamanho units=rnn_units (também poderíamos usar uma camada LSTM aqui.)

- tf.keras.layers.Dense : A camada de saída, com saídas vocab_size .

In [13]:
# Tamanho do vocabulário em caracteres
vocab_size = len(vocab)

# Dimensão dos embeddings 
embedding_dim = 256

# Número de unidades da RNN 
rnn_units = 1024

In [14]:
def build_model(vocab_size, embedding_dim, rnn_units, batch_size):
    model = tf.keras.Sequential([
        tf.keras.layers.Embedding(vocab_size, embedding_dim,
                                  batch_input_shape=[batch_size, None]),
        tf.keras.layers.GRU(rnn_units,
                            return_sequences=True,
                            stateful=True,
                            recurrent_initializer='glorot_uniform'),
        tf.keras.layers.Dense(vocab_size)
    ])
    return model

In [15]:
model = build_model(
    vocab_size=len(vocab),
    embedding_dim=embedding_dim,
    rnn_units=rnn_units,
    batch_size=BATCH_SIZE)

In [16]:
for input_example_batch, target_example_batch in dataset.take(1):
    example_batch_predictions = model(input_example_batch)
    print(example_batch_predictions.shape, "# (batch_size, sequence_length, vocab_size)")

(64, 100, 132) # (batch_size, sequence_length, vocab_size)


In [17]:
model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding (Embedding)        (64, None, 256)           33792     
_________________________________________________________________
gru (GRU)                    (64, None, 1024)          3938304   
_________________________________________________________________
dense (Dense)                (64, None, 132)           135300    
Total params: 4,107,396
Trainable params: 4,107,396
Non-trainable params: 0
_________________________________________________________________


Para obter previsões reais do modelo, precisamos de uma amostra da distribuição de saída para obter índices de caracteres reais. Esta distribuição é definida pelos logits sobre o vocabulário dos caracteres.

In [18]:
sampled_indices = tf.random.categorical(example_batch_predictions[0], num_samples=1)
sampled_indices = tf.squeeze(sampled_indices,axis=-1).numpy()

In [19]:
sampled_indices

array([ 73,  18,  32, 123,  35,  27, 125,  11, 105, 109,  90,  29, 121,
         9,   9, 113,  72,   0,  76, 124,  94, 114,  89,  32,  26,  29,
        23,  11,  84,  83, 116,  39,  73, 109,  92,  64, 128, 124,  24,
       128,  71,  20,  15,  69,  98,  46,  98,   4, 126, 126,  27,   6,
        40, 103,  32,  74, 106,  53,  38,  25,  94,  78,  99,  94,  29,
        18, 119, 109, 102,  16,  56,  38,  41,  23,  78,   0,  19, 115,
         9,  23, 102,  35,  43,  79,  68, 111, 101,  88,  33,   7,  59,
        23,  66,  21,  41, 112,  88,  54,  75,   5])

Decodifique-os para ver o texto previsto por este modelo não treinado:

In [20]:
print("Input: \n", repr("".join(idx2char[input_example_batch[1]])))
print()
print("Next Char Predictions: \n", repr("".join(idx2char[sampled_indices ])))

Input: 
 'ntes  brenhas.\nPor que é que iam, nem esperando eu desse minha primeira ganhada?\n\n― A isso, meio aco'

Next Char Predictions: 
 'r4F–IA―-çìÂCü**òq\nu—ÍóÁF?C9-´ªõMrìÉi“—:“p61nÚTÚ"‘‘A\'NäFsè[L;ÍwàÍC4úìã2aLO9w\n5ô*9ãIQxmïâÀG(d9k7OñÀ]t&'


### Treinando o modelo

Neste ponto, o problema pode ser tratado como um problema de classificação padrão. Dado o estado RNN anterior e a entrada desta etapa de tempo, prevemos a classe do próximo caractere.

A loss tf.keras.losses.sparse_categorical_crossentropy padrão funciona neste caso porque é aplicada na última dimensão das previsões.

In [21]:
def loss(labels, logits):
    return tf.keras.losses.sparse_categorical_crossentropy(labels, logits, from_logits=True)

example_batch_loss = loss(target_example_batch, example_batch_predictions)
print("Prediction shape: ", example_batch_predictions.shape, " # (batch_size, sequence_length, vocab_size)")
print("scalar_loss:      ", example_batch_loss.numpy().mean())

Prediction shape:  (64, 100, 132)  # (batch_size, sequence_length, vocab_size)
scalar_loss:       4.8817596


In [22]:
model.compile(optimizer='adam', loss=loss)

Usaremos um tf.keras.callbacks.ModelCheckpoint para garantir que os pontos de verificação sejam salvos durante o treinamento:

In [23]:
# Diretório em que os checkpoints serão salvos
checkpoint_dir = './training_checkpoints'

# Nome dos arquivos de checkpoint
checkpoint_prefix = os.path.join(checkpoint_dir, "ckpt_{epoch}")

checkpoint_callback = tf.keras.callbacks.ModelCheckpoint(
    filepath=checkpoint_prefix,
    save_weights_only=True)

#### O treinamento

Para manter o tempo de treinamento razoável, usaremos 10 épocas para treinar o modelo. Com o Colab, podemos usar uma GPU para acelerar o treinamento.

In [24]:
EPOCHS = 10

In [26]:
history = model.fit(dataset, epochs=EPOCHS, callbacks=[checkpoint_callback])

Epoch 1/10
731/731 [==============================] - 42s 57ms/step - loss: 1.3190
Epoch 2/10
731/731 [==============================] - 44s 58ms/step - loss: 1.3074
Epoch 3/10
731/731 [==============================] - 45s 60ms/step - loss: 1.2979
Epoch 4/10
731/731 [==============================] - 44s 59ms/step - loss: 1.2898
Epoch 5/10
731/731 [==============================] - 43s 58ms/step - loss: 1.2823
Epoch 6/10
731/731 [==============================] - 44s 59ms/step - loss: 1.2770
Epoch 7/10
731/731 [==============================] - 45s 60ms/step - loss: 1.2729
Epoch 8/10
731/731 [==============================] - 45s 60ms/step - loss: 1.2694
Epoch 9/10
731/731 [==============================] - 45s 60ms/step - loss: 1.2670
Epoch 10/10
731/731 [==============================] - 45s 60ms/step - loss: 1.2660


### Gerar texto

In [27]:
tf.train.latest_checkpoint(checkpoint_dir)


'./training_checkpoints/ckpt_10'

In [28]:
model = build_model(vocab_size, embedding_dim, rnn_units, batch_size=1)

model.load_weights(tf.train.latest_checkpoint(checkpoint_dir))

model.build(tf.TensorShape([1, None]))

In [29]:
model.summary()


Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_1 (Embedding)      (1, None, 256)            33792     
_________________________________________________________________
gru_1 (GRU)                  (1, None, 1024)           3938304   
_________________________________________________________________
dense_1 (Dense)              (1, None, 132)            135300    
Total params: 4,107,396
Trainable params: 4,107,396
Non-trainable params: 0
_________________________________________________________________


## O loop de previsão

In [30]:
def generate_text(model, start_string):
    
    # Número de caracteres a serem gerados
    num_generate = 1000

    # Vetorização
    input_eval = [char2idx[s] for s in start_string]
    input_eval = tf.expand_dims(input_eval, 0)

    # Lista vazia para guardar os resultados.
    text_generated = []

    temperature = 1.0

    model.reset_states()
    for i in range(num_generate):
        predictions = model(input_eval)
        
        # remove a dimensão do batch
        predictions = tf.squeeze(predictions, 0)

        # utiliza uma distribuição categórica para prever o caractere que o modelo retorna
        predictions = predictions / temperature
        predicted_id = tf.random.categorical(predictions, num_samples=1)[-1,0].numpy()

        # Passa o caractere previsto como próximo input para o modelo junto 
        # com o estado anterior da célula 
        input_eval = tf.expand_dims([predicted_id], 0)

        text_generated.append(idx2char[predicted_id])

    return (start_string + ''.join(text_generated))

In [33]:
print(generate_text(model, start_string=u"Diadorim"))

Diadorim me instruiu
se traçar o razoável, o que de mim são da burra. Dizia que se assombrar. Enfim, ainda
ele todo a pressentir-se, quanto o final desse ar!: acho que o sorriso. Antes de contar, melhor
nem assim, ela era de valor receio. O são-gente prestarão tantas compinações
calma, e próximo de meio da rua — depois pensava nele, conforme o malfeito. Via de vez de viver, com medida ainda de
um corte de presente, por meu quieto. Ninguém sabia dar
um come perfum de trato e própria a Nha
ele ao lembrar a chegar até com todo encanado. Mas dubluziu duro, do que para a varanda. Levantava
na feita ou com outras era branca; e, ainda que leve companhia de Miguilim. Mechéu com uma
palma de escapada. Pislar e cada liso. Redrata a esta, muito judiada! — de todo o tempo, de fato. Aí, toda-a-a-loirava-se a rolição, a
barberintebre vinha em lugar a donas; rezoante, mios
e Firrços, quentes e lentos, a casa-de-cria e sem forro nenhuna. Mas,
cata, menino e de, no perdão.
Outras vez pervados para a sa

## Salvar modelo

In [ ]:
model.save('modelo.h5')